In [49]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from data_cleaning import Cleaning
from model_selection import Model_selection

In [50]:
train_data = pd.read_csv("./dataset/train.csv")

In [51]:
#drop unnecessary columns
columns_to_drop = ['customer_id','name']
train_data=Cleaning.dropUnnecessaryColumns(data=train_data, columnNameList=columns_to_drop)


In [52]:
#replace unkown values with nan
train_data = Cleaning.replaceInvalidValuesWithNull(train_data)

C:\Users\91908\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [53]:
#fill unknown values
train_data = Cleaning.fill_unknown_values(train_data)

In [54]:
# replace categorical values into 1 and 0

train_data.replace(to_replace=["M","F","Y","N"],value=[1,0,1,0],inplace=True)
label_encoder = LabelEncoder()
train_data["occupation_type"] = label_encoder.fit_transform(train_data["occupation_type"])

In [55]:
# Normalization
scale_columns = ["age",'net_yearly_income',"no_of_days_employed","yearly_debt_payments","credit_limit","credit_limit_used(%)","credit_score",'occupation_type']
scaler= StandardScaler()
train_data[scale_columns] = scaler.fit_transform(train_data[scale_columns])

In [56]:
# divide data in independent and dependent columns
X = train_data.drop("credit_card_default", axis=1, inplace=False)
y= train_data["credit_card_default"]
max_score = Model_selection.best_model(X,y) 
max_score

(92.09083633627746, 'LogisticRegression is Best for this dataset')


model selection

In [57]:
max_score,x  = Model_selection.best_model(X,y) 
print( f"{x} \n F1_score = { max_score}")

LogisticRegression is Best for this dataset 
 F1_score = 92.09083633627746


In [58]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

testing data

In [59]:
main_test_data = pd.read_csv("dataset/test.csv") 

In [60]:
#drop unnecessary columns
test_data = main_test_data.copy()
columns_to_drop = ['customer_id','name']
test_data=Cleaning.dropUnnecessaryColumns(data=test_data, columnNameList=columns_to_drop)


In [61]:
test_data["gender"].unique()
test_data = Cleaning.replaceInvalidValuesWithNull(test_data)


C:\Users\91908\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [62]:
test_data = Cleaning.fill_unknown_values(test_data)

In [63]:
test_data.replace(to_replace=["M","F","Y","N"],value=[1,0,1,0],inplace=True)
test_data["gender"]=test_data["gender"].astype(int)
test_data["occupation_type"]=label_encoder.fit_transform(test_data['occupation_type'])

In [64]:
scale_columns = ["age",'net_yearly_income',"no_of_days_employed","yearly_debt_payments","credit_limit","credit_limit_used(%)","credit_score"]
scaler= StandardScaler()
test_data[scale_columns] = scaler.fit_transform(test_data[scale_columns])


In [65]:
#test data
prediction = logmodel.predict(test_data)

In [66]:
output = pd.DataFrame(prediction, index=main_test_data.customer_id,columns=["credit_card_default"])
output.to_csv("output_Prediction.csv")

In [67]:
from keras import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

X_train, X_val, y_train, y_val= train_test_split(X,y,test_size = 0.3,random_state=100)
X_val.shape

(13659, 16)

In [68]:
callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
ann_model= Sequential()
ann_model.add(Dense(32,activation="relu"))
ann_model.add(Dropout(0.3))
ann_model(Dense(16,activation="relu"))
ann_model.add(Dropout(0.2))
ann_model.add(Dense(8,activation="sigmoid"))
ann_model.add(Dropout(0.2))
ann_model.add(Dense(1,activation="sigmoid"))

ann_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
ann_model.fit(X_train,y_train,batch_size=63,epochs=100,verbose=1,validation_data=(X_val,y_val),callbacks=callback)
prediction = ann_model.predict(X_val)
Ann_score = (100*f1_score(y_val, prediction,average='macro'))
Ann_score

TypeError: Inputs to a layer should be tensors. Got: <keras.layers.core.Dense object at 0x00000299B4949610>